In [36]:
rm( list=ls() )  #Borro todos los objetos
gc()   #Garbage Collection

library("data.table")
library("rpart")
library("parallel")

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,710298,38.0,1321401,70.6,1321401,70.6
Vcells,1386509,10.6,89979595,686.5,126785534,967.3


In [37]:
#ksemillas  <- c(262139, 394943, 638819, 819031, 902761) #reemplazar por las propias semillas

ksemillas  <- c( 162896, 298841, 198981, 269607, 312548, 487610, 482911, 687314, 742337, 382084
                ,232222, 503457, 391252, 466578, 669404, 137551, 147833, 235500, 341086, 643133)

In [38]:
# particionar agrega una columna llamada fold a un dataset que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30), agrupa=clase_ternaria, seed=semilla)   crea una particion 70, 30 

particionar  <- function( data,  division, agrupa="",  campo="fold", start=1, seed=NA )
{
  if( !is.na(seed) )   set.seed( seed )

  bloque  <- unlist( mapply(  function(x,y) { rep( y, x )} ,   division,  seq( from=start, length.out=length(division) )  ) )  

  data[ , (campo) :=  sample( rep( bloque, ceiling(.N/length(bloque))) )[1:.N],
          by= agrupa ]
}

In [39]:
ArbolEstimarGanancia  <- function( semilla, param_basicos )
{
  #particiono estratificadamente el dataset
  particionar( dataset, division=c(7,3), agrupa="clase_ternaria", seed= semilla )  #Cambiar por la primer semilla de cada uno !

  #genero el modelo
  modelo  <- rpart("clase_ternaria ~ .",     #quiero predecir clase_ternaria a partir del resto
                   data= dataset[ fold==1],  #fold==1  es training,  el 70% de los datos
                   xval= 0,
                   control= param_basicos )  #aqui van los parametros del arbol

  #aplico el modelo a los datos de testing
  prediccion  <- predict( modelo,   #el modelo que genere recien
                          dataset[ fold==2],  #fold==2  es testing, el 30% de los datos
                          type= "prob") #type= "prob"  es que devuelva la probabilidad

  #prediccion es una matriz con TRES columnas, llamadas "BAJA+1", "BAJA+2"  y "CONTINUA"
  #cada columna es el vector de probabilidades 

  #calculo la ganancia en testing  qu es fold==2
  ganancia_test  <- dataset[ fold==2, 
                             sum( ifelse( prediccion[, "BAJA+2"]  >  0.025,
                                         ifelse( clase_ternaria=="BAJA+2", 117000, -3000 ),
                                         0 ) )]
                                         
  #escalo la ganancia como si fuera todo el dataset
  ganancia_test_normalizada  <-  ganancia_test / 0.3

  return( list( "testing"=       dataset[ fold==2, .N],
                "testing_pos"=   dataset[ fold==2 & clase_ternaria=="BAJA+2", .N],
                "envios"=        dataset[ fold==2 , sum( prediccion[ , "BAJA+2"] > 0.025)],
                "aciertos"=      dataset[ fold==2, sum( prediccion[ , "BAJA+2"] > 0.025 & clase_ternaria=="BAJA+2" )],
                "ganancia_test"= ganancia_test_normalizada ))
}

In [40]:
#apunto el working directory al directorio donde esta ubicado el script
setwd(".\\") 
getwd() #solo para validar


[1] "d:/Onedrive/Austral/Cursos/08 - Laboratorio de Implementacion 1/labo2023v/src/rpart"

In [41]:
#cargo los datos
dataset  <- fread("./datasets/dataset_pequeno.csv")
#trabajo solo con los datos con clase, es decir 202107
dataset  <- dataset[ clase_ternaria!= "" ]

In [42]:
param_basicos_ext  <- list( "cp"=          -1,  #complejidad minima
                        "minsplit"=   900,  #minima cantidad de registros en un nodo para hacer el split
                        "minbucket"=  440,  #minima cantidad de registros en una hoja
                        "maxdepth"=     5 ) #profundidad máxima del arbol
param_basicos_ext       

$cp
[1] -1

$minsplit
[1] 900

$minbucket
[1] 440

$maxdepth
[1] 5

In [43]:
#Un solo llamado, con la semilla 17
ArbolEstimarGanancia( 17, param_basicos_ext )   

$testing
[1] 49405

$testing_pos
[1] 380

$envios
[1] 3899

$aciertos
[1] 231

$ganancia_test
[1] 53410000

In [44]:
#la funcion mcmapply  llama a la funcion ArbolEstimarGanancia  tantas veces como valores tenga el vector  ksemillas
salidas  <- mcmapply( ArbolEstimarGanancia, 
                      ksemillas,   #paso el vector de semillas, que debe ser el primer parametro de la funcion ArbolEstimarGanancia
                      MoreArgs= list( param_basicos_ext),  #aqui paso el segundo parametro
                      SIMPLIFY= FALSE,
                      mc.cores= 1 )  #se puede subir a 5 si posee Linux o Mac OS

In [45]:
#muestro la lista de las salidas en testing para la particion realizada con cada semilla
salidas

[[1]]
[[1]]$testing
[1] 49406

[[1]]$testing_pos
[1] 381

[[1]]$envios
[1] 4102

[[1]]$aciertos
[1] 247

[[1]]$ganancia_test
[1] 57780000


[[2]]
[[2]]$testing
[1] 49403

[[2]]$testing_pos
[1] 380

[[2]]$envios
[1] 3875

[[2]]$aciertos
[1] 232

[[2]]$ganancia_test
[1] 54050000


[[3]]
[[3]]$testing
[1] 49406

[[3]]$testing_pos
[1] 380

[[3]]$envios
[1] 3524

[[3]]$aciertos
[1] 216

[[3]]$ganancia_test
[1] 51160000


[[4]]
[[4]]$testing
[1] 49402

[[4]]$testing_pos
[1] 379

[[4]]$envios
[1] 3275

[[4]]$aciertos
[1] 210

[[4]]$ganancia_test
[1] 51250000


[[5]]
[[5]]$testing
[1] 49404

[[5]]$testing_pos
[1] 381

[[5]]$envios
[1] 3650

[[5]]$aciertos
[1] 235

[[5]]$ganancia_test
[1] 57500000


[[6]]
[[6]]$testing
[1] 49405

[[6]]$testing_pos
[1] 380

[[6]]$envios
[1] 3511

[[6]]$aciertos
[1] 211

[[6]]$ganancia_test
[1] 49290000


[[7]]
[[7]]$testing
[1] 49402

[[7]]$testing_pos
[1] 381

[[7]]$envios
[1] 3340

[[7]]$aciertos
[1] 216

[[7]]$ganancia_test
[1] 5.3e+07


[[8]]
[[8]]$testing
[1] 49405

[[8]]$testing_pos
[1] 380

[[8]]$envios
[1] 3820

[[8]]$aciertos
[1] 222

[[8]]$ganancia_test
[1] 50600000


[[9]]
[[9]]$testing
[1] 49404

[[9]]$testing_pos
[1] 379

[[9]]$envios
[1] 4107

[[9]]$aciertos
[1] 231

[[9]]$ganancia_test
[1] 51330000


[[10]]
[[10]]$testing
[1] 49405

[[10]]$testing_pos
[1] 380

[[10]]$envios
[1] 3538

[[10]]$aciertos
[1] 221

[[10]]$ganancia_test
[1] 53020000


[[11]]
[[11]]$testing
[1] 49405

[[11]]$testing_pos
[1] 379

[[11]]$envios
[1] 3494

[[11]]$aciertos
[1] 215

[[11]]$ganancia_test
[1] 51060000


[[12]]
[[12]]$testing
[1] 49401

[[12]]$testing_pos
[1] 380

[[12]]$envios
[1] 3567

[[12]]$aciertos
[1] 225

[[12]]$ganancia_test
[1] 54330000


[[13]]
[[13]]$testing
[1] 49407

[[13]]$testing_pos
[1] 381

[[13]]$envios
[1] 4149

[[13]]$aciertos
[1] 239

[[13]]$ganancia_test
[1] 54110000


[[14]]
[[14]]$testing
[1] 49405

[[14]]$testing_pos
[1] 381

[[14]]$envios
[1] 3865

[[14]]$aciertos
[1] 233

[[14]]$ganancia_test
[1] 54550000


[[15]]
[[15]]$testing
[1] 49405

[[15]]$testing_pos
[1] 381

[[15]]$envios
[1] 3738

[[15]]$aciertos
[1] 235

[[15]]$ganancia_test
[1] 56620000


[[16]]
[[16]]$testing
[1] 49405

[[16]]$testing_pos
[1] 381

[[16]]$envios
[1] 3547

[[16]]$aciertos
[1] 221

[[16]]$ganancia_test
[1] 52930000


[[17]]
[[17]]$testing
[1] 49403

[[17]]$testing_pos
[1] 380

[[17]]$envios
[1] 3681

[[17]]$aciertos
[1] 212

[[17]]$ganancia_test
[1] 47990000


[[18]]
[[18]]$testing
[1] 49406

[[18]]$testing_pos
[1] 381

[[18]]$envios
[1] 3554

[[18]]$aciertos
[1] 217

[[18]]$ganancia_test
[1] 51260000


[[19]]
[[19]]$testing
[1] 49405

[[19]]$testing_pos
[1] 381

[[19]]$envios
[1] 3530

[[19]]$aciertos
[1] 224

[[19]]$ganancia_test
[1] 54300000


[[20]]
[[20]]$testing
[1] 49403

[[20]]$testing_pos
[1] 379

[[20]]$envios
[1] 4422

[[20]]$aciertos
[1] 235

[[20]]$ganancia_test
[1] 49780000

In [46]:
#paso la lista a vector
tb_salida  <- rbindlist(salidas)
tb_salida

testing,testing_pos,envios,aciertos,ganancia_test
<int>,<int>,<int>,<int>,<dbl>
49406,381,4102,247,57780000
49403,380,3875,232,54050000
49406,380,3524,216,51160000
49402,379,3275,210,51250000
49404,381,3650,235,57500000
49405,380,3511,211,49290000
49402,381,3340,216,53000000
49405,380,3820,222,50600000
49404,379,4107,231,51330000


In [48]:
#finalmente calculo la media (promedio)  de las ganancias
#calculo todos los promedios
tb_salida[  , mean( ganancia_test ) ]

[1] 52795500

In [49]:
#desvio estandar Distribucion Binomial   sqrt( n * p * (1-p) )
tb_salida[  , lapply( .SD, mean ) ]

testing,testing_pos,envios,aciertos,ganancia_test
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
49404.35,380.25,3714.45,224.85,52795500
